In [17]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import seaborn as sns
import xlrd
import cv2
import os
import csv
import re
import pandas as pd
import array as arr
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

#Read csv files and convert it to excel files
read_file = pd.read_csv (r'kalawatibeddevicecetail17april.csv')
read_file.to_excel (r'file1.xlsx', index = None, header=True)

read_file = pd.read_csv (r'kalawatirefinicubbox_17april.csv')
read_file.to_excel (r'file2.xlsx', index = None, header=True)

#Open Excel file
file1 = pd.read_excel('file1.xlsx')
file2 = pd.read_excel('file2.xlsx')
file3 = pd.read_excel('KalawatiWithISt.xlsx')

In [18]:
from datetime import datetime
from dateutil import tz
from datetime import timedelta  

In [23]:
txt_1 = pd.read_csv('finalFile_3.txt', sep=" ",header=None)

In [24]:
txt_1 = txt_1.rename(columns={0: 'Video (Title)', 1: 'Start'})

In [25]:
txt_2 = pd.read_csv('finalFile_4.txt', sep=" ",header=None)

In [26]:
txt_2 = txt_2.rename(columns={0: 'Video (Title)', 1: 'Start'})

In [27]:
txt = pd.concat([txt_1,txt_2])

In [28]:
txt.drop_duplicates(subset=['Video (Title)','Start'],inplace=True)

In [30]:
txt.dropna(inplace=True)

In [31]:
txt.count()

Video (Title)    1050
Start            1050
dtype: int64

In [32]:
def extract_time(x):
    
    
    t = str(x).split("_")[1]
    if t != '720p':
        from_zone = tz.tzutc()
        to_zone = tz.tzlocal()
        test = t.split("-")[0] +"-" + t.split("-")[1] + "-" + t.split("-")[2] + " " + ((t.split("-")[3]).split(".")[0] +":"+ (t.split("-")[3]).split(".")[1] + ":"+ (t.split("-")[3]).split(".")[2])
        utc = datetime.strptime(test, '%Y-%m-%d %H:%M:%S')
        central = utc.astimezone(to_zone)
        central = pd.to_datetime(central)+timedelta(seconds=19800)
    else:
        t = str(x).split("_")[2]
        from_zone = tz.tzutc()
        to_zone = tz.tzlocal()
        test = t.split("-")[0] +"-" + t.split("-")[1] + "-" + t.split("-")[2] + " " + ((t.split("-")[3]).split(".")[0] +":"+ (t.split("-")[3]).split(".")[1] + ":"+ (t.split("-")[3]).split(".")[2])
        utc = datetime.strptime(test, '%Y-%m-%d %H:%M:%S')
        central = utc.astimezone(to_zone)
        central = pd.to_datetime(central)+timedelta(seconds=19800)
        
    return ((str(pd.to_datetime(central)).split("+")[0]))

def extract_neo(x):
    s = str(x).split("_")[0]
    
    return (s.split(" ")[-1].split(".")[0].lower())

def extract_seq(x):
    s = str(x).split("_")[-1]
    
    return s.split(".")[0]

def to_split_0(x):
    x = str(x)
    return x.split(".")[0]
def to_split_1(x):
    try:
        x = str(x)
        return x.split(".")[1]
    except:
        pass

def date_extract(x):
    return x.split(" ")[0]

def time_join(x,y):
    try:
    
        to_zone = tz.tzlocal()
        test = str(x)+" "+str(y)
        utc = datetime.strptime(test, '%Y-%m-%d %H:%M:%S')
        central = utc.astimezone(to_zone)
        central = pd.to_datetime(central)+timedelta(seconds=19800)
        return central
    except:
        to_zone = tz.tzlocal()
        test = str(x)+" "+str(y)
        utc = datetime.strptime(test, '%Y-%m-%d %H.%M.%S')
        central = utc.astimezone(to_zone)
        central = pd.to_datetime(central)+timedelta(seconds=19800)
        return central
    
    
        
    
    
    
    

In [33]:
file3['NEO'] = file3['Video (Title)'].apply(extract_neo)
file3['Start_time_NEO'] = file3['Video (Title)'].apply(extract_time)

In [34]:
t = pd.merge(file3,txt,on='Video (Title)',how='left')

In [35]:
t['start_date'] = t['Start_time_NEO'].apply(date_extract)
t.dropna(subset=['Start'],inplace=True)

In [36]:
t['NEO_START_TIME'] = t.apply(lambda x: time_join(x['start_date'], x['Start']), axis=1)

In [37]:
file3 = t.copy()

In [38]:
file3['Tag'] = file3['Video (Title)'].apply(extract_seq)
file3['to_min'] = file3['To (min.sec)'].apply(to_split_1)
file3['From_min'] = file3['From (min.sec)'].apply(to_split_1)
file3['to_Hour'] = file3['To (min.sec)'].apply(to_split_0)
file3['From_Hour'] = file3['From (min.sec)'].apply(to_split_0)
file3['to_min'] = file3['to_min'].astype(float)
file3['From_min'] = file3['From_min'].astype(float)

In [39]:
def replace_NIL(x):
    if x == 'NIL' or x =='camera angle changed':
        return '0'
    else:
        return x

In [40]:
file3['to_Hour'] = file3['to_Hour'].apply(replace_NIL)
file3['to_Hour'] = file3['to_Hour'].astype(float)

In [41]:
file3['From_Hour'] = file3['From_Hour'].apply(replace_NIL)
file3['From_Hour'] = file3['From_Hour'].astype(float)

In [42]:
df = pd.DataFrame()

In [43]:
for i in file3['NEO'].unique():
    print(1)
    x = file3[file3['NEO']==i]
    print(2)
    for j in x['Video (Title)'].unique():
        #print(j)
        try:
            print(j)
            y = x[x['Video (Title)']==j]
            #for k in x['tags'].unique():
            #z = x[x['tags']==k]
            y['Duration'] = (y['to_Hour']*60 + y['to_min']) - (y['From_Hour']*60 + y['From_min'])
            #y['Duration_shift'] = y['Duration'].shift(1,axis=0)
            y['cummulative_duration'] = y['Duration'].cumsum()
            df = df.append(y,ignore_index=True)
            
            
            print(i,j,np.nansum(y['Duration']))
        
        except:
            continue

1
2
NEO102.stream_720p_2020-01-21-07.57.03.617-UTC_1.mp4


C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


neo102 NEO102.stream_720p_2020-01-21-07.57.03.617-UTC_1.mp4 74.0
NEO102.stream_720p_2020-01-21-08.14.13.220-UTC_2.mp4
neo102 NEO102.stream_720p_2020-01-21-08.14.13.220-UTC_2.mp4 606.0
NEO102.stream_720p_2020-01-09-08.29.10.842-UTC_0.mp4
neo102 NEO102.stream_720p_2020-01-09-08.29.10.842-UTC_0.mp4 0.0
NEO102.stream_720p_2020-01-09-08.29.50.542-UTC_0.mp4
neo102 NEO102.stream_720p_2020-01-09-08.29.50.542-UTC_0.mp4 0.0
NEO102.stream_720p_2020-01-09-08.30.37.766-UTC_0.mp4
neo102 NEO102.stream_720p_2020-01-09-08.30.37.766-UTC_0.mp4 28.0
NEO102.stream_720p_2020-01-09-08.32.03.141-UTC_0.mp4
neo102 NEO102.stream_720p_2020-01-09-08.32.03.141-UTC_0.mp4 0.0
NEO102.stream_720p_2020-01-09-08.43.25.974-UTC_0.mp4
neo102 NEO102.stream_720p_2020-01-09-08.43.25.974-UTC_0.mp4 0.0
NEO102.stream_720p_2020-01-09-08.49.41.211-UTC_0.mp4
neo102 NEO102.stream_720p_2020-01-09-08.49.41.211-UTC_0.mp4 113.0
NEO102.stream_720p_2020-01-09-08.52.39.283-UTC_0.mp4
neo102 NEO102.stream_720p_2020-01-09-08.52.39.283-UTC_0.mp

neo102 NEO102.stream_720p_2020-01-15-19.35.07.219-UTC_4.mp4 0.0
NEO102.stream_720p_2020-01-15-19.35.07.219-UTC_5.mp4
neo102 NEO102.stream_720p_2020-01-15-19.35.07.219-UTC_5.mp4 201.0
NEO102.stream_720p_2020-01-15-19.35.07.219-UTC_6.mp4
neo102 NEO102.stream_720p_2020-01-15-19.35.07.219-UTC_6.mp4 0.0
NEO102.stream_720p_2020-01-15-19.35.07.219-UTC_7.mp4
neo102 NEO102.stream_720p_2020-01-15-19.35.07.219-UTC_7.mp4 137.0
NEO102.stream_720p_2020-01-15-19.35.07.219-UTC_8.mp4
neo102 NEO102.stream_720p_2020-01-15-19.35.07.219-UTC_8.mp4 76.0
NEO102.stream_720p_2020-01-15-19.35.07.219-UTC_9.mp4
neo102 NEO102.stream_720p_2020-01-15-19.35.07.219-UTC_9.mp4 246.0
NEO102.stream_720p_2020-01-15-19.35.07.219-UTC_10.mp4
neo102 NEO102.stream_720p_2020-01-15-19.35.07.219-UTC_10.mp4 0.0
NEO102.stream_720p_2020-01-15-19.35.07.219-UTC_11.mp4
neo102 NEO102.stream_720p_2020-01-15-19.35.07.219-UTC_11.mp4 10.0
NEO102.stream_720p_2020-01-15-19.35.07.219-UTC_12.mp4
neo102 NEO102.stream_720p_2020-01-15-19.35.07.219-U

neo102 NEO102.stream_720p_2020-01-16-17.49.14.549-UTC_0.mp4 0.0
NEO102.stream_720p_2020-01-16-17.50.04.495-UTC_0.mp4
neo102 NEO102.stream_720p_2020-01-16-17.50.04.495-UTC_0.mp4 0.0
NEO102.stream_720p_2020-01-16-17.52.34.238-UTC_0.mp4
neo102 NEO102.stream_720p_2020-01-16-17.52.34.238-UTC_0.mp4 0.0
NEO102.stream_720p_2020-01-16-17.53.34.609-UTC_0.mp4
neo102 NEO102.stream_720p_2020-01-16-17.53.34.609-UTC_0.mp4 0.0
NEO102.stream_720p_2020-01-16-17.54.34.249-UTC_0.mp4
neo102 NEO102.stream_720p_2020-01-16-17.54.34.249-UTC_0.mp4 0.0
NEO102.stream_720p_2020-01-16-17.55.24.620-UTC_0.mp4
neo102 NEO102.stream_720p_2020-01-16-17.55.24.620-UTC_0.mp4 0.0
NEO102.stream_720p_2020-01-16-17.57.04.630-UTC_0.mp4
neo102 NEO102.stream_720p_2020-01-16-17.57.04.630-UTC_0.mp4 0.0
NEO102.stream_720p_2020-01-16-17.58.04.188-UTC_0.mp4
neo102 NEO102.stream_720p_2020-01-16-17.58.04.188-UTC_0.mp4 0.0
NEO102.stream_720p_2020-01-16-17.59.04.498-UTC_0.mp4
neo102 NEO102.stream_720p_2020-01-16-17.59.04.498-UTC_0.mp4 0.0


...

neo101 NEO101.stream_720p_2020-01-07-09.27.22.054-UTC_0.mp4 0.0
NEO101.stream_720p_2020-01-07-09.27.22.054-UTC_1.mp4
neo101 NEO101.stream_720p_2020-01-07-09.27.22.054-UTC_1.mp4 138.0
NEO101.stream_720p_2020-01-07-10.44.53.609-UTC_0.mp4
neo101 NEO101.stream_720p_2020-01-07-10.44.53.609-UTC_0.mp4 0.0
NEO101.stream_720p_2020-01-07-10.44.53.609-UTC_1.mp4
neo101 NEO101.stream_720p_2020-01-07-10.44.53.609-UTC_1.mp4 0.0
NEO101.stream_720p_2020-01-07-11.01.12.817-UTC_0.mp4
neo101 NEO101.stream_720p_2020-01-07-11.01.12.817-UTC_0.mp4 0.0
NEO101.stream_720p_2020-01-07-11.01.12.817-UTC_1.mp4
neo101 NEO101.stream_720p_2020-01-07-11.01.12.817-UTC_1.mp4 0.0
NEO101.stream_720p_2020-01-07-11.01.12.817-UTC_2.mp4
neo101 NEO101.stream_720p_2020-01-07-11.01.12.817-UTC_2.mp4 0.0
NEO101.stream_720p_2020-01-07-11.01.12.817-UTC_3.mp4
neo101 NEO101.stream_720p_2020-01-07-11.01.12.817-UTC_3.mp4 236.0
NEO101.stream_720p_2020-01-07-11.01.12.817-UTC_4.mp4
neo101 NEO101.stream_720p_2020-01-07-11.01.12.817-UTC_4.mp4 

neo101 NEO101.stream_720p_2020-01-09-13.57.40.398-UTC_0.mp4 0.0
NEO101.stream_720p_2020-01-09-13.57.40.398-UTC_1.mp4
neo101 NEO101.stream_720p_2020-01-09-13.57.40.398-UTC_1.mp4 0.0
NEO101.stream_720p_2020-01-09-14.01.48.656-UTC_0.mp4
neo101 NEO101.stream_720p_2020-01-09-14.01.48.656-UTC_0.mp4 0.0
NEO101.stream_720p_2020-01-09-14.11.48.028-UTC_0.mp4
neo101 NEO101.stream_720p_2020-01-09-14.11.48.028-UTC_0.mp4 0.0
NEO101.stream_720p_2020-01-09-14.12.57.639-UTC_0.mp4
neo101 NEO101.stream_720p_2020-01-09-14.12.57.639-UTC_0.mp4 0.0
NEO101.stream_720p_2020-01-09-14.24.17.809-UTC_0.mp4
neo101 NEO101.stream_720p_2020-01-09-14.24.17.809-UTC_0.mp4 0.0
NEO101.stream_720p_2020-01-09-14.28.52.551-UTC_0.mp4
neo101 NEO101.stream_720p_2020-01-09-14.28.52.551-UTC_0.mp4 0.0
NEO101.stream_720p_2020-01-09-14.41.17.024-UTC_0.mp4
neo101 NEO101.stream_720p_2020-01-09-14.41.17.024-UTC_0.mp4 0.0
NEO101.stream_720p_2020-01-09-14.46.22.246-UTC_0.mp4
neo101 NEO101.stream_720p_2020-01-09-14.46.22.246-UTC_0.mp4 0.0


neo106 NEO106.stream_720p_2020-01-05-07.12.26.889-UTC_1.mp4 1164.0
NEO106.stream_720p_2020-01-05-08.21.04.472-UTC_0.mp4
neo106 NEO106.stream_720p_2020-01-05-08.21.04.472-UTC_0.mp4 3.0
NEO106.stream_720p_2020-01-05-08.21.24.487-UTC_0.mp4
neo106 NEO106.stream_720p_2020-01-05-08.21.24.487-UTC_0.mp4 197.0
NEO106.stream_720p_2020-01-05-08.25.06.404-UTC_0.mp4
neo106 NEO106.stream_720p_2020-01-05-08.25.06.404-UTC_0.mp4 0.0
NEO106.stream_720p_2020-01-05-08.25.30.277-UTC_0.mp4
neo106 NEO106.stream_720p_2020-01-05-08.25.30.277-UTC_0.mp4 2068.0
NEO106.stream_720p_2020-01-05-08.25.30.277-UTC_1.mp4
neo106 NEO106.stream_720p_2020-01-05-08.25.30.277-UTC_1.mp4 3499.0
NEO106.stream_720p_2020-01-05-08.25.30.277-UTC_2.mp4
neo106 NEO106.stream_720p_2020-01-05-08.25.30.277-UTC_2.mp4 3144.0
NEO106.stream_720p_2020-01-05-10.59.02.063-UTC_0.mp4
neo106 NEO106.stream_720p_2020-01-05-10.59.02.063-UTC_0.mp4 50.0
NEO106.stream_720p_2020-01-05-10.59.02.063-UTC_1.mp4
neo106 NEO106.stream_720p_2020-01-05-10.59.02.063

neotiny102 NEOTINY102.stream_720p_2020-03-06-09.39.03.371-UTC_0.mp4 1255.0
NEOTINY102.stream_720p_2020-03-06-09.39.03.371-UTC_1.mp4
neotiny102 NEOTINY102.stream_720p_2020-03-06-09.39.03.371-UTC_1.mp4 1374.0
NEOTINY102.stream_720p_2020-03-06-10.24.33.478-UTC_0.mp4
neotiny102 NEOTINY102.stream_720p_2020-03-06-10.24.33.478-UTC_0.mp4 654.0
NEOTINY102.stream_720p_2020-03-06-11.09.51.156-UTC_0.mp4
neotiny102 NEOTINY102.stream_720p_2020-03-06-11.09.51.156-UTC_0.mp4 12.0
NEOTINY102.stream_720p_2020-03-06-11.11.20.563-UTC_0.mp4
neotiny102 NEOTINY102.stream_720p_2020-03-06-11.11.20.563-UTC_0.mp4 11.0
NEOTINY102.stream_720p_2020-03-06-11.25.57.460-UTC_0.mp4
neotiny102 NEOTINY102.stream_720p_2020-03-06-11.25.57.460-UTC_0.mp4 1547.0
NEOTINY102.stream_720p_2020-03-06-11.53.17.392-UTC_0.mp4
neotiny102 NEOTINY102.stream_720p_2020-03-06-11.53.17.392-UTC_0.mp4 62.0
NEOTINY102.stream_720p_2020-03-08-04.40.47.927-UTC_0.mp4
neotiny102 NEOTINY102.stream_720p_2020-03-08-04.40.47.927-UTC_0.mp4 0.0
NEOTINY102.

neotiny102 NEOTINY102.stream_720p_2020-02-24-11.07.13.045-UTC_0.mp4 0.0
NEOTINY102.stream_720p_2020-02-24-11.48.02.650-UTC_1.mp4
neotiny102 NEOTINY102.stream_720p_2020-02-24-11.48.02.650-UTC_1.mp4 1913.0
NEOTINY102.stream_720p_2020-02-24-11.48.02.650-UTC_3.mp4
neotiny102 NEOTINY102.stream_720p_2020-02-24-11.48.02.650-UTC_3.mp4 169.0
NEOTINY102.stream_720p_2020-02-24-11.48.02.650-UTC_4.mp4
neotiny102 NEOTINY102.stream_720p_2020-02-24-11.48.02.650-UTC_4.mp4 0.0
NEOTINY102.stream_720p_2020-02-24-16.39.58.806-UTC_0.mp4
neotiny102 NEOTINY102.stream_720p_2020-02-24-16.39.58.806-UTC_0.mp4 0.0
NEOTINY102.stream_720p_2020-02-24-16.39.58.806-UTC_1.mp4
neotiny102 NEOTINY102.stream_720p_2020-02-24-16.39.58.806-UTC_1.mp4 0.0
NEOTINY102.stream_720p_2020-02-24-16.39.58.806-UTC_2.mp4
neotiny102 NEOTINY102.stream_720p_2020-02-24-16.39.58.806-UTC_2.mp4 55.0
NEOTINY102.stream_720p_2020-02-24-16.39.58.806-UTC_4.mp4
neotiny102 NEOTINY102.stream_720p_2020-02-24-16.39.58.806-UTC_4.mp4 0.0
NEOTINY102.stream_7

In [44]:
df['END'] = 0
df['cummulative_duration'].fillna(0,inplace=True)

In [45]:
for i in range(len(df)):
    df['END'].iloc[i] = pd.to_datetime(df['NEO_START_TIME'].iloc[i]) + timedelta(seconds=df['cummulative_duration'].iloc[i])

C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [46]:
df['START'] = 0
df['Duration'].fillna(0,inplace=True)

In [47]:
for i in range(len(df)):
    df['START'].iloc[i] = pd.to_datetime(df['END'].iloc[i]) - timedelta(seconds=df['Duration'].iloc[i])

In [48]:
df.to_csv('times_kalawati_5.csv')

In [53]:
df[df['Video (Title)']=='NEO102.stream_720p_2020-01-16-19.19.44.838-UTC_0.mp4']

,2020-01-25 00:27:50,UHID,Video (Title),Total Duration of video file (min.sec),From (min.sec),To (min.sec),Procedure or manipulation (Name),Label,Duration (seconds),Procedure_IST_Time,From_IST_Time,NEO,Start_time_NEO,Start,start_date,NEO_START_TIME,Tag,to_min,From_min,to_Hour,From_Hour,Duration,cummulative_duration,END,START
351,912,NaN,NEO102.stream_720p_2020-01-16-19.19.44.838-UTC...,0.33,0.00,0.33,Feeding through OG tube,-,Sequence not complete,2020-01-17 00:49:44,2020-01-17 00:49:44,neo102,2020-01-17 00:49:44,19.19.44,2020-01-17,2020-01-18 00:49:44+05:30,0,33.0,0.0,0.0,0.0,33.0,33.0,2020-01-18 00:50:17+05:30,2020-01-18 00:49:44+05:30


In [51]:
txt[txt['Video (Title)']=='NEO101.stream_720p_2020-01-06-19.00.26.977-UTC_6.mp4']

,Video (Title),Start
19,NEO101.stream_720p_2020-01-06-19.00.26.977-UTC...,22:00:00


In [56]:
df.head(30)

,2020-01-25 00:27:50,UHID,Video (Title),Total Duration of video file (min.sec),From (min.sec),To (min.sec),Procedure or manipulation (Name),Label,Duration (seconds),Procedure_IST_Time,From_IST_Time,NEO,Start_time_NEO,Start,start_date,NEO_START_TIME,Tag,to_min,From_min,to_Hour,From_Hour,Duration,cummulative_duration,END,START
0,0,191001977.0,NEO102.stream_720p_2020-01-21-07.57.03.617-UTC...,13.57,5.20,5.57,Diaper change,Diaper_change,NaN,NaN,NaN,neo102,2020-01-21 13:27:03,08:00:00,2020-01-21,2020-01-21 13:30:00+05:30,1,57.0,20.0,5.0,5.0,37.0,37.0,2020-01-21 13:30:37+05:30,2020-01-21 13:30:00+05:30
1,1343,NaN,NEO102.stream_720p_2020-01-21-07.57.03.617-UTC...,13.57,5.19,5.56,Diaper change,Diaper_change,NaN,2020-01-21 13:35:17,2020-01-21 13:29:58,neo102,2020-01-21 13:27:03,08:00:00,2020-01-21,2020-01-21 13:30:00+05:30,1,56.0,19.0,5.0,5.0,37.0,74.0,2020-01-21 13:31:14+05:30,2020-01-21 13:30:37+05:30
2,1,191001977.0,NEO102.stream_720p_2020-01-21-08.14.13.220-UTC...,59.59,9.07,10.58,Feeding,Feeding,NaN,NaN,NaN,neo102,2020-01-21 13:44:13,09:00:00,2020-01-21,2020-01-21 14:30:00+05:30,2,58.0,7.0,10.0,9.0,111.0,111.0,2020-01-21 14:31:51+05:30,2020-01-21 14:30:00+05:30
3,2,191001977.0,NEO102.stream_720p_2020-01-21-08.14.13.220-UTC...,59.59,11.22,12.48,Patting,Patting,NaN,NaN,NaN,neo102,2020-01-21 13:44:13,09:00:00,2020-01-21,2020-01-21 14:30:00+05:30,2,48.0,22.0,12.0,11.0,86.0,197.0,2020-01-21 14:33:17+05:30,2020-01-21 14:31:51+05:30
4,3,191001977.0,NEO102.stream_720p_2020-01-21-08.14.13.220-UTC...,59.59,12.54,13.39,Patting,Patting,NaN,NaN,NaN,neo102,2020-01-21 13:44:13,09:00:00,2020-01-21,2020-01-21 14:30:00+05:30,2,39.0,54.0,13.0,12.0,45.0,242.0,2020-01-21 14:34:02+05:30,2020-01-21 14:33:17+05:30
5,4,191001977.0,NEO102.stream_720p_2020-01-21-08.14.13.220-UTC...,59.59,13.42,14.02,Position change,Position_change,NaN,NaN,NaN,neo102,2020-01-21 13:44:13,09:00:00,2020-01-21,2020-01-21 14:30:00+05:30,2,2.0,42.0,14.0,13.0,20.0,262.0,2020-01-21 14:34:22+05:30,2020-01-21 14:34:02+05:30
6,5,191001977.0,NEO102.stream_720p_2020-01-21-08.14.13.220-UTC...,59.59,14.20,14.24,Remove spo2 probe,Other,NaN,NaN,NaN,neo102,2020-01-21 13:44:13,09:00:00,2020-01-21,2020-01-21 14:30:00+05:30,2,24.0,20.0,14.0,14.0,4.0,266.0,2020-01-21 14:34:26+05:30,2020-01-21 14:34:22+05:30
7,6,191001977.0,NEO102.stream_720p_2020-01-21-08.14.13.220-UTC...,59.59,14.25,15.07,Apply spo2 probe,Other,NaN,NaN,NaN,neo102,2020-01-21 13:44:13,09:00:00,2020-01-21,2020-01-21 14:30:00+05:30,2,7.0,25.0,15.0,14.0,42.0,308.0,2020-01-21 14:35:08+05:30,2020-01-21 14:34:26+05:30
8,7,191001977.0,NEO102.stream_720p_2020-01-21-08.14.13.220-UTC...,59.59,27.31,27.35,Position change,Position_change,NaN,NaN,NaN,neo102,2020-01-21 13:44:13,09:00:00,2020-01-21,2020-01-21 14:30:00+05:30,2,35.0,31.0,27.0,27.0,4.0,312.0,2020-01-21 14:35:12+05:30,2020-01-21 14:35:08+05:30
9,1346,NaN,NEO102.stream_720p_2020-01-21-08.14.13.220-UTC...,59.59,9.08,10.57,Feeding through OG tube,Feeding,NaN,2020-01-21 14:38:40,2020-01-21 14:29:32,neo102,2020-01-21 13:44:13,09:00:00,2020-01-21,2020-01-21 14:30:00+05:30,2,57.0,8.0,10.0,9.0,109.0,421.0,2020-01-21 14:37:01+05:30,2020-01-21 14:35:12+05:30
